In [5]:
!pip freeze | grep "beautifulsoup"
!pip freeze | grep "requests"
#pip install beautifulsoup
#pip install requests

beautifulsoup4==4.6.3
requests==2.23.0
requests-oauthlib==1.3.0


In [44]:
#@title スクレイピングするURLを指定する。<br>例)https://hatsutora.com/doctor-yellow-unkou-2021-11/
BASEURL = "https://hatsutora.com/doctor-yellow-unkou" #@param {type:"string"}


In [59]:
#@title DebugFlgを指定する。True/False
DebugFlg = "False" #@param {type:"string"}

In [48]:
#@title IsDebug デバックモードかどうか
def IsDebug() -> bool:
    #print(f"DebugFlg is {DebugFlg}")
    return DebugFlg.lower() == "true"

In [51]:
#@title PrintDebug デバックモードでprintする
def PrintDebug(output) -> None:
    if IsDebug():
        print(output)

In [49]:
from datetime import datetime

today = datetime.now()
# print(today.year)
# print(today.month)
url = f"{BASEURL}-{today.year}-{today.month}/"

In [ ]:
from bs4 import BeautifulSoup
import requests

request = requests.get(url)
soup = BeautifulSoup(request.content, "html.parser")
#<h3><span id="_202111">ドクターイエロー 2021年11月の予想運行日</span></h3>の直下に運行予定日のtableがあり、最初のtableが運行予定表となる模様
tables = soup.find_all("table")
table = tables[0]
PrintDebug(table)
rows = table.findAll('tr')

In [64]:
isHeaderSkipped = False
row_index = 0
datakey = ("Date", "Operations")
timetable = [datakey]
for row in rows:

    # 先頭行はヘッダー行なので読み飛ばす
    if not isHeaderSkipped:
        isHeaderSkipped = True
        row_index += 1
        continue
    
    PrintDebug(f"row_index:{row_index}")

    column_index = 0
    timetable_row = []
    for cell in row.findAll('td'):

        if column_index == 2:
            continue

        # timetable_row[datakey[column_index]] = cell.get_text()
        # timetable_row[column_index] = cell.get_text()
        timetable_row.append(cell.get_text())
        PrintDebug(f"column_index:{column_index}, 値:{cell.get_text()}")
        # if column_index == 0:
        #     timetable_row[datakey[column_index]] = cell.get_text()
        # elif column_index == 1:
        #     # operations = cell.get_text().split('・')
        #     # PrintDebug(f"operations:{operations}")
        column_index += 1

    timetable.append(timetable_row)
    row_index += 1

In [65]:
timetable

[('Date', 'Operations'),
 ['11月5日 (金)', 'のぞみ・下り (東京駅 → 博多駅)'],
 ['11月6日 (土)', 'のぞみ・上り (博多駅 → 東京駅)'],
 ['11月16日 (火)', 'のぞみ・下り (東京駅 → 博多駅)'],
 ['11月17日 (水)', 'のぞみ・上り (博多駅 → 東京駅)'],
 ['11月20日 (土)', '【試運転】のぞみダイヤ・下り (東京駅 → 博多駅)'],
 ['11月22日 (月)', '【試運転】のぞみダイヤ・上り (博多駅 → 東京駅)'],
 ['11月26日 (金)', 'のぞみ・下り (東京駅 → 博多駅)'],
 ['11月27日 (土)', 'のぞみ・上り (博多駅 → 東京駅)'],
 ['11月27日 (土)', '【回送】のぞみダイヤ・下り (東京駅 → 博多駅)'],
 ['11月30日 (火)', '【回送】のぞみダイヤ・上り (博多駅 → 東京駅)']]